## Load tree via uproot

In [ ]:
import uproot
import awkward as ak
import numpy as np

In [ ]:
fileName = '/udrive/staff/dnoonan/Skims/TTGamma_SingleLept_2016_skim.root'
_ttg_file = uproot.open(fileName)

In [ ]:
#look at the objects within the root file (accessible with the list of keys)
_ttg_file.keys()

In [ ]:
#Get the TTree named 'Events'
_ttg_tree = _ttg_file['Events']

In [ ]:
#look at the list of branches within the Events tree
_ttg_tree.keys()

In [ ]:
#Load arrays for the pt and eta of jets in the first 10k events
arrays = _ttg_tree.arrays(['Jet_pt','Jet_eta'],entry_stop=10000)

In [ ]:
arrays['Jet_eta']

In [ ]:
arrays['Jet_pt']

In [ ]:
jetSelect = ((arrays['Jet_pt']>30) & 
             (abs(arrays['Jet_eta'])<2.4))

In [ ]:
jetSelect

In [ ]:
#keep only good jets
goodJetPt = arrays['Jet_pt'][jetSelect]

In [ ]:
#count number of jets in each event
ak.num(arrays['Jet_pt'])

In [ ]:
#count number of jets passing selection in each event
ak.num(goodJetPt)

#### Make plot of pt and eta of all jets, and selected jets

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(ak.flatten(arrays['Jet_pt']),bins=np.linspace(0,100,101),label='All Jets')
plt.hist(ak.flatten(goodJetPt),bins=np.linspace(0,100,101),label='Good Jets')
plt.xlabel('Jet $p_T$')
plt.legend();

## Load data with Coffea

Use NanoEvents within Coffea to read and interpret file, loading branches

In [ ]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema


In [ ]:
#the NanoAODSchema needs to be adjusted, to remove cross references to FSRPhotons
class SkimmedSchema(NanoAODSchema):
    def __init__(self, base_form):
        base_form["contents"].pop("Muon_fsrPhotonIdx", None)
        super().__init__(base_form)

fileName = '/udrive/staff/dnoonan/Skims/TTGamma_SingleLept_2016_skim.root'

events = NanoEventsFactory.from_root(fileName, schemaclass=SkimmedSchema,entry_stop=10000).events()


In NanoEvents, objects and properties of those objects 

In [ ]:
#printout of jet pt array
events.Jet.pt

In [ ]:
#printout of jet eta array
events.Jet.eta

In [ ]:
jetSelect = ((events.Jet.pt>30) & 
             (abs(events.Jet.eta)<2.4))

In [ ]:
goodJets = events.Jet[jetSelect]
goodJets.pt

In [ ]:
#histogram of Jet pt for all jets and only selected jets
plt.hist(ak.flatten(events.Jet.pt),bins=np.linspace(0,100,101),label='All Jets')
plt.hist(ak.flatten(goodJets.pt),bins=np.linspace(0,100,101),label='Good Jets')
plt.legend()
plt.xlabel('Jet $p_T$');

In [ ]:
#histogram of Jet eta for all jets and only selected jets
plt.hist(ak.flatten(events.Jet.eta),bins=np.linspace(-5,5,101),label='All Jets')
plt.hist(ak.flatten(goodJets.eta),bins=np.linspace(-5,5,101),label='Good Jets')
plt.legend()
plt.xlabel('Jet $\eta$');

### Additional steps
 - Make a plot the pt of the highest pt jet in the event. _(Hint, this will be the first in each event, and awkward arrays support slicing in the same way that a numpy array do, ex: array[startIdx:stopIdx], or multiple axis can be sliced by specifying both ranges with a comma)_
 - Add an additional selection cut to the jet requirements, requiring all good jets pass the tight ID requirement (accessible with the .isTight method of the Jet class)
 - Apply the deep CSV b tagger, to select only the subset of good jets with a b-taggind discriminant value (btagDeepB branch) of ≥ 0.6321. Make histograms comparing the distribution of pT and eta for b-tagged jets vs non b-tagged jets
 - Make histograms counting the number of jets and the number of b-tagged jets (hint: the ak.num function will be used).  
 - Of the 10000 events we looked at, how many have at least 4 jets, and at least 1 b-tagged jet?  How many if we if we require at least 2 b-tagged jets instead?
 - Select good muons, requiring pt >30, |eta| < 2.4, tight ID, and relative isolation (pfRelIso04_all) < 0.15
 - Make a histogram of the muon pt and eta
 - Select events with exactly 1 good muon, at least 4 jets, and at least one b-tagged jet.  How many events are there?
 - For these events, make histograms of the pt and eta of the leading photons.
 - Select good electrons, requiring pt >35, |eta| < 2.1, and tight ID, and make histograms of electron pt and eta
 - Select events with exactly 1 good electron, 0 good muons, ≥4 jets, ≥1 b-tagged jet, and plot the pt and eta of the leading photon
 - Make the same plots for a W+gamma sample